In [1]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_md")
text = "The quick brown fox jumps over the lazy dog."
doc = nlp(text)

In [2]:
displacy.serve(doc, style="dep")


c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\displacy\__init__.py:108: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)


127.0.0.1 - - [11/Dec/2025 08:41:41] "GET / HTTP/1.1" 200 7538



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...



127.0.0.1 - - [11/Dec/2025 08:41:41] "GET /favicon.ico HTTP/1.1" 200 7538


Shutting down server on port 5000.


#### Câu hỏi:
1. Từ nào là gốc (ROOT) của câu?
2. jumps có những từ phụ thuộc (dependent) nào? Các quan hệ đó
là gì?
3. fox là head của những từ nào?

#### Trả lời:
1. Từ gốc (ROOT) của câu là "jumps" (VERB).

2. Các từ phụ thuộc của "jumps" và quan hệ

- fox -> quan hệ nsubj (subject của động từ)

- over-> quan hệ prep (giới từ phụ thuộc động từ "jumps")

3. "fox" là head của những từ nào?

- The → quan hệ det (mạo từ)

- quick → quan hệ amod (tính từ bổ nghĩa danh từ)

- brown → quan hệ amod (tính từ bổ nghĩa danh từ)

In [ ]:
text = "Apple is looking at buying U.K. startup for $1 billion"
doc = nlp(text)
# In ra thông tin của từng token
print(f"{'TEXT':<12} | {'DEP':<10} | {'HEAD TEXT':<12} | {'HEAD POS':<8} | {'CHILDREN'}")
print("-" * 70)
for token in doc:
# Trích xuất các thuộc tính
    children = [child.text for child in token.children] 
    
    print(f"{token.text:<12} | {token.dep_:<10} | {token.head.text:<12} | {token.head.pos_:<8} | {children}")

TEXT         | DEP        | HEAD TEXT    | HEAD POS | CHILDREN
----------------------------------------------------------------------
Apple        | nsubj      | looking      | VERB     | []
is           | aux        | looking      | VERB     | []
looking      | ROOT       | looking      | VERB     | ['Apple', 'is', 'at']
at           | prep       | looking      | VERB     | ['buying']
buying       | pcomp      | at           | ADP      | ['startup']
U.K.         | compound   | startup      | NOUN     | []
startup      | dobj       | buying       | VERB     | ['U.K.', 'for']
for          | prep       | startup      | NOUN     | ['billion']
$            | quantmod   | billion      | NUM      | []
1            | compound   | billion      | NUM      | []
billion      | pobj       | for          | ADP      | ['$', '1']


In [ ]:
text = "The cat chased the mouse and the dog watched them."
doc = nlp(text)
for token in doc:
# Chỉ tìm các động từ
    if token.pos_ == "VERB":
        verb = token.text
        subject = ""
        obj = ""
# Tìm chủ ngữ (nsubj) và tân ngữ (dobj) trong các con của động từ
        for child in token.children:
            if child.dep_ == "nsubj":
                subject = child.text
            if child.dep_ == "dobj":
                obj = child.text

        if subject and obj:
            print(f"Found Triplet: ({subject}, {verb}, {obj})")

Found Triplet: (cat, chased, mouse)
Found Triplet: (dog, watched, them)


In [ ]:
text = "The big, fluffy white cat is sleeping on the warm mat."
doc = nlp(text)
for token in doc:
    if token.pos_ == "NOUN":
        adjectives = []
        for child in token.children:
            if child.dep_ == "amod":
                adjectives.append(child.text)
            if adjectives:
                print(f"Danh từ '{token.text}' được bổ nghĩa bởi các tính từ: {adjectives}")

Danh từ 'cat' được bổ nghĩa bởi các tính từ: ['big']
Danh từ 'cat' được bổ nghĩa bởi các tính từ: ['big']
Danh từ 'cat' được bổ nghĩa bởi các tính từ: ['big', 'fluffy']
Danh từ 'cat' được bổ nghĩa bởi các tính từ: ['big', 'fluffy', 'white']
Danh từ 'mat' được bổ nghĩa bởi các tính từ: ['warm']


### Bài 1:

In [4]:
def find_main_verb(doc):
   
    for token in doc:
        if token.dep_ == "ROOT" and token.pos_ in ("VERB", "AUX"):
            return token

    for token in doc:
        if token.dep_ == "ROOT":
            return token

    return None

doc = nlp("Hanoi is the capital of Vietnam.")
main_verb = find_main_verb(doc)

print("Động từ chính:", main_verb)

Động từ chính: is


### Bài 2:

In [ ]:
def extract_noun_chunks(doc):
   
    modifier_deps = {"det", "amod", "compound", "poss", "nummod"}
    chunks = []
    seen = set()

    for token in doc:
        if token.pos_ in ("NOUN", "PROPN"):
            start = token.i
            end = token.i + 1  

            for left in token.lefts:
                if left.dep_ in modifier_deps:
                    start = min(start, left.i)

            span = doc[start:end]
            key = (span.start, span.end)
            if key not in seen:
                seen.add(key)
                chunks.append(span)

    return chunks


doc = nlp("The beautiful old city of Hanoi is the capital of Vietnam.")
for chunk in extract_noun_chunks(doc):
    print(chunk.text)

The beautiful old city
Hanoi
the capital
Vietnam


### Bài 3:

In [ ]:
def get_path_to_root(token):

    path = []
    current = token

    while True:
        path.append(current)
        if current.dep_ == "ROOT" or current.head == current:
            break
        current = current.head

    return path

doc = nlp("The beautiful city of Hanoi is the capital of Vietnam.")

hanoi = [t for t in doc if t.text == "Hanoi"][0]

path = get_path_to_root(hanoi)
print(" -> ".join(f"{t.text}({t.dep_})" for t in path))

Hanoi(pobj) -> of(prep) -> city(nsubj) -> is(ROOT)
